In [2]:
!pip install torch tensorboard --quiet

# Install Hugging Face libraries
!pip install  --upgrade transformers datasets accelerate evaluate bitsandbytes --quiet

#FlashAttention only supports Ampere GPUs or newer. #NEED A100 IN GOOGLE COLAB
#!pip install -U transformers
# !pip install -U flash-attn --no-build-isolation --quiet


! pip install peft --quiet
! pip install datasets trl ninja packaging --quiet

# Uncomment only if you're using A100 GPU
#!pip install flash-attn --no-build-isolation
!pip install diffusers safetensors  --quiet

%pip install -U wandb

Note: you may need to restart the kernel to use updated packages.


In [4]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import (
    LoraConfig,
    PeftModel,
    prepare_model_for_kbit_training,
    get_peft_model,
)
import os, torch, wandb
from datasets import load_dataset
from trl import SFTTrainer, setup_chat_format

In [5]:
from datasets import load_dataset, DatasetDict

In [124]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch
device = torch.device('cuda')
model_path = 'phamhai/Llama-3.2-1B-Instruct-Frog'
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path).to('cuda')

In [125]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

hf_token = user_secrets.get_secret("HF_TOKEN2")

login(token = hf_token)

wb_token = user_secrets.get_secret("wandb")

wandb.login(key=wb_token)
run = wandb.init(
    project='Fine-tune Llama 3 8B on SQL dataset', 
    job_type="training", 
    anonymous="allow"
)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


In [126]:
base_model = "phamhai/Llama-3.2-1B-Instruct-Frog"
dataset_path = "/kaggle/input/sql-dataset/train_dataset.json"
new_model = "llama-3.2-1b-chat-sql3"

In [127]:
torch_dtype = torch.float16

In [128]:
# QLoRA config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.float16
)

# Load model
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.float16
    #attn_implementation=attn_implementation
)
tokenizer = AutoTokenizer.from_pretrained(base_model,use_fast=True)

In [129]:
# LoRA config
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['up_proj', 'down_proj', 'gate_proj', 'k_proj', 'q_proj', 'v_proj', 'o_proj']
)
model = get_peft_model(model, peft_config)

In [130]:
# train_set = load_dataset('huyhoangt2201/jidouka3.1',split="train")
# test_set = load_dataset('huyhoangt2201/jidouka3.1',split='test')
dataset_train = load_dataset("huyhoangt2201/jidouka3.1", split='train[:90%]')
dataset_val = load_dataset("huyhoangt2201/jidouka3.1", split='train[-10%:]')
dataset = DatasetDict({
    'train': dataset_train,
    'validation': dataset_val
})
dataset.save_to_disk("completed_train_dataset")

Saving the dataset (0/1 shards):   0%|          | 0/1132 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/126 [00:00<?, ? examples/s]

In [131]:
training_arguments = TrainingArguments(
    output_dir=new_model,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=2,
    optim="paged_adamw_32bit",
    num_train_epochs=10,
    eval_strategy="steps",
    eval_steps=0.2,
    logging_steps=1,
    warmup_steps=10,
    logging_strategy="steps",
    learning_rate=2e-4,
    fp16=False,
    bf16=False,
    group_by_length=True,
    report_to="wandb"
)

In [57]:
dataset['train'][0]

{'context': 'Có 1 bảng tên jidouka cần truy vấn. Bảng cần truy vấn bao gồm các cột: \nid: số thứ tự của hàng (int);\ntên_cải_tiến: tên của tác phẩm cải tiến (str);\nloại_hình_công_việc: loại hình công việc mà tác phẩm đó cải tiến? (str) (ví dụ: Xử lí database, nhập thông tin, tối ưu quy trình làm việc,...) ;\ncông_cụ: Công cụ hỗ trợ để thực hiện mục đích cải tiến (str) (ví dụ: Python, Excel, Visual Studio Code, ...);\nmô_tả: Mô tả cụ thể chi tiết cải tiến (str) (ví dụ: từng bước thực hiện cải tiến đó) ;\nsản_phẩm: sản phẩm đầu ra của tác phẩm cải tiến đó (str) (ví dụ: file csv, file xlsx, ....);\ntác_giả: người đóng góp, nhân viên trong công ty, người đã tạo ra cải tiến đó (str)  ;\nbộ_phận: Phòng ban làm việc của tác_giả, thường gọi là dc nào đó  (str) (ví dụ: dc1, dc2, dc3, dcd, souko,...);\nsố_giờ: số lượng giờ mà nhờ việc áp dụng cải tiến tiết kiệm được (int);\nsố_công_việc_áp_dụng: số công việc trong công ty mà cải tiến đó đã giúp được (int)\nthời_điểm_ra_mắt: Thời điểm công cụ nà

In [58]:
schema = """
Bảng jidouka: 
    id: số thứ tự của hàng (int)
    tên_cải_tiến: tên của tác phẩm cải tiến (str)
    loại_hình_công_việc: loại hình công việc mà tác phẩm đó cải tiến? (str) (ví dụ: Xử lí database, nhập thông tin, tối ưu quy trình làm việc,...) 
    công_cụ: Công cụ hỗ trợ để thực hiện mục đích cải tiến (str) (ví dụ: Python, Excel, Visual Studio Code, ...);
    mô_tả: Mô tả cụ thể chi tiết cải tiến (str) (ví dụ: từng bước thực hiện cải tiến đó) ;
    sản_phẩm: sản phẩm đầu ra của tác phẩm cải tiến đó (str) (ví dụ: file csv, file xlsx, ....);
    tác_giả: người đóng góp, nhân viên trong công ty, người đã tạo ra cải tiến đó (str)  ;
    bộ_phận: Phòng ban làm việc của tác_giả, thường gọi là dc nào đó  (str) (ví dụ: dc1, dc2, dc3, dcd, souko,...);
    số_giờ: số lượng giờ mà nhờ việc áp dụng cải tiến tiết kiệm được (int);
    số_công_việc_áp_dụng: số công việc trong công ty mà cải tiến đó đã giúp được (int)
    thời_điểm_ra_mắt: Thời điểm công cụ này ra mắt (str) (ví dụ: 2024-10-11, 2024-10-09,...);
    thông_tin_thêm: Đường link tài liệu hướng dẫn (powerpoint, video) sử dụng cải tiến, công cụ của cải tiến (str)',
"""

In [80]:
dataset['train'][0]

{'context': 'Có 1 bảng tên jidouka cần truy vấn. Bảng cần truy vấn bao gồm các cột: \nid: số thứ tự của hàng (int);\ntên_cải_tiến: tên của tác phẩm cải tiến (str);\nloại_hình_công_việc: loại hình công việc mà tác phẩm đó cải tiến? (str) (ví dụ: Xử lí database, nhập thông tin, tối ưu quy trình làm việc,...) ;\ncông_cụ: Công cụ hỗ trợ để thực hiện mục đích cải tiến (str) (ví dụ: Python, Excel, Visual Studio Code, ...);\nmô_tả: Mô tả cụ thể chi tiết cải tiến (str) (ví dụ: từng bước thực hiện cải tiến đó) ;\nsản_phẩm: sản phẩm đầu ra của tác phẩm cải tiến đó (str) (ví dụ: file csv, file xlsx, ....);\ntác_giả: người đóng góp, nhân viên trong công ty, người đã tạo ra cải tiến đó (str)  ;\nbộ_phận: Phòng ban làm việc của tác_giả, thường gọi là dc nào đó  (str) (ví dụ: dc1, dc2, dc3, dcd, souko,...);\nsố_giờ: số lượng giờ mà nhờ việc áp dụng cải tiến tiết kiệm được (int);\nsố_công_việc_áp_dụng: số công việc trong công ty mà cải tiến đó đã giúp được (int)\nthời_điểm_ra_mắt: Thời điểm công cụ nà

In [95]:
def format_context_to_schema_dataset(sample):
    schema = """
Bảng jidouka: 
    id: số thứ tự của hàng (int)
    tên_cải_tiến: tên của tác phẩm cải tiến (str)
    loại_hình_công_việc: loại hình công việc mà tác phẩm đó cải tiến? (str) (ví dụ: Xử lí database, nhập thông tin, tối ưu quy trình làm việc,...) 
    công_cụ: Công cụ hỗ trợ để thực hiện mục đích cải tiến (str) (ví dụ: Python, Excel, Visual Studio Code, ...);
    mô_tả: Mô tả cụ thể chi tiết cải tiến (str) (ví dụ: từng bước thực hiện cải tiến đó) ;
    sản_phẩm: sản phẩm đầu ra của tác phẩm cải tiến đó (str) (ví dụ: file csv, file xlsx, ....);
    tác_giả: người đóng góp, nhân viên trong công ty, người đã tạo ra cải tiến đó (str)  ;
    bộ_phận: Phòng ban làm việc của tác_giả, thường gọi là dc nào đó  (str) (ví dụ: dc1, dc2, dc3, dcd, souko,...);
    số_giờ: số lượng giờ mà nhờ việc áp dụng cải tiến tiết kiệm được (int);
    số_công_việc_áp_dụng: số công việc trong công ty mà cải tiến đó đã giúp được (int)
    thời_điểm_ra_mắt: Thời điểm công cụ này ra mắt (str) (ví dụ: 2024-10-11, 2024-10-09,...);
    thông_tin_thêm: Đường link tài liệu hướng dẫn (powerpoint, video) sử dụng cải tiến, công cụ của cải tiến (str)',
"""
    sample['context'] = schema
    return sample

In [96]:
dataset_change_train = dataset['train'].map(format_context_to_schema_dataset)
dataset_change_valid = dataset['validation'].map(format_context_to_schema_dataset)

Map:   0%|          | 0/1132 [00:00<?, ? examples/s]

Map:   0%|          | 0/126 [00:00<?, ? examples/s]

In [98]:
dataset_change_train[0]

{'context': "\nBảng jidouka: \n    id: số thứ tự của hàng (int)\n    tên_cải_tiến: tên của tác phẩm cải tiến (str)\n    loại_hình_công_việc: loại hình công việc mà tác phẩm đó cải tiến? (str) (ví dụ: Xử lí database, nhập thông tin, tối ưu quy trình làm việc,...) \n    công_cụ: Công cụ hỗ trợ để thực hiện mục đích cải tiến (str) (ví dụ: Python, Excel, Visual Studio Code, ...);\n    mô_tả: Mô tả cụ thể chi tiết cải tiến (str) (ví dụ: từng bước thực hiện cải tiến đó) ;\n    sản_phẩm: sản phẩm đầu ra của tác phẩm cải tiến đó (str) (ví dụ: file csv, file xlsx, ....);\n    tác_giả: người đóng góp, nhân viên trong công ty, người đã tạo ra cải tiến đó (str)  ;\n    bộ_phận: Phòng ban làm việc của tác_giả, thường gọi là dc nào đó  (str) (ví dụ: dc1, dc2, dc3, dcd, souko,...);\n    số_giờ: số lượng giờ mà nhờ việc áp dụng cải tiến tiết kiệm được (int);\n    số_công_việc_áp_dụng: số công việc trong công ty mà cải tiến đó đã giúp được (int)\n    thời_điểm_ra_mắt: Thời điểm công cụ này ra mắt (str)

In [110]:
system_message = """Bạn là một trợ lí ảo thông minh hiểu biết về SQL. User sẽ hỏi bạn câu hỏi bằng tiếng Việt và bạn sẽ tạo ra câu truy vấn SQL dựa vào SCHEMA được cho dưới đây.
SCHEMA:
{schema}
"""
def format_data_template(sample):
    return {
      "messages": [
          {"role":"system", "content": system_message.format(schema=sample['context'])},
          {"role":"user", "content":sample['question']},
          {"role":"assistant","content":sample['answer']}
      ]
  }

In [106]:
dataset['train']

Dataset({
    features: ['context', 'question', 'answer'],
    num_rows: 1132
})

In [135]:
dataset_train = dataset_change_train.map(format_data_template, remove_columns = ['context','question','answer'], batched=False)
dataset_valid = dataset_change_valid.map(format_data_template, remove_columns = ['context', 'question', 'answer'], batched=False)

In [136]:
dataset_train

Dataset({
    features: ['messages'],
    num_rows: 1132
})

In [137]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset_train,
    eval_dataset=dataset_valid,
    peft_config=peft_config,
    max_seq_length=512,
    tokenizer=tokenizer,
    args=training_arguments,
    dataset_text_field='messages',
    packing= False,
)

/opt/conda/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length, dataset_text_field. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:300: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:328: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/1132 [00:00<?, ? examples/s]

Map:   0%|          | 0/126 [00:00<?, ? examples/s]

In [138]:
eot = "<|eot_id|>"
eot_id = tokenizer.convert_tokens_to_ids(eot)
tokenizer.pad_token = eot
tokenizer.pad_token_id = eot_id

In [ ]:
trainer.train()

Step,Training Loss,Validation Loss
1132,0.031200,0.055496
2264,0.031000,0.056441
3396,0.024600,0.057439
4528,0.021200,0.065154


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


In [ ]:
wandb.finish()
model.config.use_cache = True

In [ ]:
new_model

In [ ]:
trainer.model.save_pretrained(new_model)
trainer.model.push_to_hub(new_model, use_temp_dir=False)

In [ ]:
new_model = '/kaggle/working/llama-3.2-1b-chat-sql3'
base_model = 'phamhai/Llama-3.2-1B-Instruct-Frog'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(base_model)

base_model_reload = AutoModelForCausalLM.from_pretrained(
        base_model,
        return_dict=True,
        low_cpu_mem_usage=True,
        torch_dtype=torch.float16,
        device_map="auto",
        trust_remote_code=True,
)

# base_model_reload, tokenizer = setup_chat_format(base_model_reload, tokenizer)

# Merge adapter with base model
merge_model = PeftModel.from_pretrained(base_model_reload, new_model)

merge_model = model.merge_and_unload()

In [ ]:
merge_model.save_pretrained('llama-3.2-1b-chat-sql-base2')
tokenizer.save_pretrained('llama-3.2-1b-chat-sql-base2')

In [ ]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

hf_token = user_secrets.get_secret("HF_TOKEN")
login(token = hf_token)

In [ ]:
merge_model.push_to_hub("llama-3.2-1b-chat-sql-base2", use_temp_dir=False)
tokenizer.push_to_hub("llama-3.2-1b-chat-sql-base2", use_temp_dir=False)

In [53]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch
device = torch.device('cuda')
model_path = 'huyhoangt2201/llama-3.2-1b-chat-sql-base1'
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path).to('cuda')

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.41k [00:00<?, ?B/s]

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors:   0%|          | 0.00/1.03G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

In [54]:
prompt_template=""" Bạn là một trợ lí về truy vấn SQL. Dựa vào thông tin có trong schema dưới đây, hãy tạo ra câu lệnh sql để truy vấn ra kết quả phù hợp.
Câu lệnh sql phải có chính xác tên của các cột trong bảng trong schema, tuyệt đối không gen ra tên cột, bảng không có trong schem.
Nếu một câu hỏi không liên quan tới schema, hãy thừa nhận rằng mình không biết, thay vì cố trả lời sai. Hãy đưa ra câu trả lời logic và thích hợp nhất.
              Schema:
              <START_OF_SCHEMA>
              create table jidouka (
                id int not null primary key auto_increment,
                tên_cải_tiến nvarchar(50) not null,
                loại_hình_công_việc nvarchar(20) not null,
                công_cụ nvarchar(50) not null,
                mô_tả_cải_tiến text,
                phần_mềm nvarchar(50),
                sản_phảm nvarchar(50),
                người_đóng_góp nvarchar(30),
                bộ_phận(dc) nvarchar(10),
                số_lượng_công_việc_được_cải_tiến int,
                số_giờ_tiết_kiệm_được int,
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                thông_tin_thêm text)
              <END_OF_SCHEMA>
                """

In [55]:
from typing import List, Dict
class ContextAwareChatbot:
    def __init__(self,prompt, max_history: int = 5):
        self.model = model
        self.tokenizer = tokenizer 
        self.max_history = max_history
        self.conversation_history: List[Dict[str, str]] = []
        self.prompt=prompt
    def _build_prompt(self) -> str:
        # Build context from history

        return self.prompt

    def _clean_response(self, response: str) -> str:
        # Clean up the generated response
        response = response.split("Assistant:")[-1].strip()
        # Stop at any new "Human:" or "Assistant:" markers
        if "Human:" in response:
            response = response.split("Human:")[0].strip()
        return response

    def chat(self, user_input: str) -> str:
        # Generate the contextualized prompt
        prompt = self._build_prompt()

#         # Generate response
#         response = self.pipeline(
#             prompt,
#             return_full_text=False,
#             clean_up_tokenization_spaces=True
#         )[0]['generated_text']

#         # Clean the response
#         cleaned_response = self._clean_response(response)
        messages =[
            {'role':'system',
             'content':prompt}
            ,
            {'role':'user',
             'content':user_input}
        ]
        tokenized_chat = self.tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors='pt')
        outputs = self.model.generate(tokenized_chat, max_new_tokens=256).to('cuda')
        bot_response = self.tokenizer.decode(outputs[0])
        bot_response = bot_response.split('<|start_header_id|>assistant<|end_header_id|>')
        bot_response = bot_response[1].strip()[:-10]
        # Update conversation history
        self.conversation_history.append({
            'human': user_input,
            'assistant': bot_response
        })

        return bot_response

    def get_history(self) -> List[Dict[str, str]]:
        return self.conversation_history

    def clear_history(self):
        self.conversation_history = []

# 4. Create chatbot instance
chatbot = ContextAwareChatbot(prompt_template)

# 5. Example usage function
def chat_session():
    print("Chatbot initialized. Type 'exit' to end the conversation, 'clear' to clear history.")

    while True:
        user_input = input("\nYou: ").strip()

        if user_input.lower() == 'exit':
            print("Goodbye!")
            break
        elif user_input.lower() == 'clear':
            chatbot.clear_history()
            print("Conversation history cleared!")
            continue

        response = chatbot.chat(user_input)
        print(f"\nAssistant: {response}")

# 6. Example of how to use
if __name__ == "__main__":
    chat_session()

Chatbot initialized. Type 'exit' to end the conversation, 'clear' to clear history.



You:  xin chào


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:2097: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(



Assistant: điểm tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiết tiế


You:  bạn bị sao thế


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



Assistant: Điểm cụ thể cần cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể cụ thể


KeyboardInterrupt: Interrupted by user